<a href="https://colab.research.google.com/github/MahdiFaourr/MahdiFaourr/blob/main/SarsCovid_ctscan_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opendatasets

In [ ]:
!pip install gradio==3.14.0

In [ ]:
import opendatasets as od
import os
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import gradio as gr
from PIL import Image
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,Flatten,MaxPooling2D,Input,Conv2D
from tensorflow.keras.metrics import Precision
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
# Load the dataset from Kaggle
od.download("https://www.kaggle.com/datasets/plameneduardo/sarscov2-ctscan-dataset?select=COVID")

In [ ]:
root_dir="/content/sarscov2-ctscan-dataset"
classes=['COVID','non-COVID']
for class_name in classes:
  train_dir=os.path.join(root_dir,"train",class_name)
  os.makedirs(train_dir,exist_ok=True)
  valid_dir=os.path.join(root_dir,"valid",class_name)
  os.makedirs(valid_dir,exist_ok=True)
COVID_images=list(os.listdir(os.path.join(root_dir,"COVID")))
non_COVID_images=list(os.listdir(os.path.join(root_dir,"non-COVID")))

In [ ]:
covid_train_images,covid_valid_images=train_test_split(COVID_images,test_size=0.1,random_state=42)
for img in covid_train_images:
  source_path_1=os.path.join(root_dir,'COVID',img)
  destination_path_1=os.path.join(root_dir,'train','COVID',img)
  shutil.move(source_path_1,destination_path_1)
for image in covid_valid_images:
  source_path_2=os.path.join(root_dir,'COVID',image)
  destination_path_2=os.path.join(root_dir,'valid','COVID',image)
  shutil.move(source_path_2,destination_path_2)

In [ ]:
non_covid_train_images,non_covid_valid_images=train_test_split(non_COVID_images,test_size=0.1,random_state=42)
for img in non_covid_train_images:
  source_path_1=os.path.join(root_dir,'non-COVID',img)
  destination_path_1=os.path.join(root_dir,'train','non-COVID',img)
  shutil.move(source_path_1,destination_path_1)
for image in non_covid_valid_images:
  source_path_2=os.path.join(root_dir,'non-COVID',image)
  destination_path_2=os.path.join(root_dir,'valid','non-COVID',image)
  shutil.move(source_path_2,destination_path_2)


In [ ]:
# Define the data generator and specify preprocessing/augmentation options
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=False,
    fill_mode='nearest'
)

# Create a generator for training data
train_generator = datagen.flow_from_directory(
    "/content/sarscov2-ctscan-dataset/train",
    batch_size=32,
    class_mode='binary')

# Create a generator for validation data (optional)
validation_generator = datagen.flow_from_directory(
    "/content/sarscov2-ctscan-dataset/valid",
    batch_size=32,
    class_mode='binary'
)

Found 2232 images belonging to 2 classes.
Found 249 images belonging to 2 classes.


In [ ]:
# Define input shape
input_shape = (256, 256, 3)  #  Input images are 256x256 RGB images

# Define input layer
inputs = Input(shape=input_shape)

# Convolutional layers
layer_1= Conv2D(12, kernel_size=(3, 3), activation='relu')(inputs)
layer_2 =MaxPooling2D(pool_size=(2, 2))(layer_1)
layer_3 = Conv2D(24, kernel_size=(3, 3), activation='relu')(layer_2)
layer_4 = MaxPooling2D(pool_size=(2, 2))(layer_3)

# Flatten layer
layer_5= Flatten()(layer_4)

# Dense layers
layer_6 = Dense(100, activation='relu')(layer_5)
outputs = Dense(1, activation='sigmoid')(layer_6)

# Create the model
model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',Precision()])

# Print model summary
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 254, 254, 12)      336       
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 127, 12)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 24)      2616      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 62, 62, 24)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 92256)             0     

In [ ]:
# Define a checkpoint callback to save the best
checkpoint = ModelCheckpoint('best_weights.h5', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
# Train your model with the callback
history=model.fit(train_generator,validation_data=(validation_generator),epochs=30,batch_size=len(train_generator),callbacks=[checkpoint])

In [ ]:
# Load the best weights into your model
model.load_weights('best_weights.h5')

In [ ]:
# Evaluate the model on valid_generator
model.evaluate(validation_generator)

8/8 [==============================] - 8s 966ms/step - loss: 0.3760 - accuracy: 0.8434 - precision: 0.8443


[0.37595340609550476, 0.8433734774589539, 0.8442623019218445]

In [ ]:
# `history.history` is a dictionary containing the training and validation metrics
# Plotting accuracy
plt.subplot(2, 1, 1)  # 2 rows, 1 column, plot 1 (top subplot)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plotting precision
plt.subplot(2, 1, 2)  # 2 rows, 1 column, plot 2 (bottom subplot)
plt.plot(history.history['precision'])
plt.plot(history.history['val_precision'])
plt.title('Model Precision')
plt.xlabel('Epoch')
plt.ylabel('Precision')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()  # Adjust subplot layout to avoid overlap

# Show plot
plt.show()

In [ ]:
# Save the model as an HDF5 file
model.save('sarascov2_ctscan_model.h5')

In [ ]:
def diagnosis(img_path):
    # Convert the numpy array to a PIL Image object
    img = Image.open(img_path)

    # Convert grayscale image to RGB
    if img.mode != 'RGB':
        img = img.convert('RGB')

    # Resize the image to match the model's expected input size
    img = img.resize((256, 256))

    # Normalize the image data
    img_array = np.array(img) / 255.0

    # Expand the dimensions to match the model's expected input shape
    img_array = np.expand_dims(img_array, axis=0)

    # Use the model to predict
    prediction = model.predict(img_array)
    #return prediction
    # Check if prediction is positive or negative
    if prediction <= 0.5:
        return 'Covid Presence: positive.'
    else:
        return 'Covid Presence: negative.'


In [ ]:
# Example usage 1
image="/content/sarscov2-ctscan-dataset/valid/non-COVID/Non-Covid (2).png"
diagnosis(image)

1/1 [==============================] - 0s 36ms/step


'Covid Presence: negative.'

In [ ]:
# Example usage 2
image="/content/sarscov2-ctscan-dataset/valid/non-COVID/Non-Covid (562).png"
diagnosis(image)

1/1 [==============================] - 0s 60ms/step


'Covid Presence: negative.'

In [ ]:
# Example usage 3
image="/content/sarscov2-ctscan-dataset/valid/COVID/Covid (1137).png"
diagnosis(image)

1/1 [==============================] - 0s 37ms/step


'Covid Presence: positive.'

In [ ]:
# Example usage 4
image='/content/sarscov2-ctscan-dataset/valid/COVID/Covid (344).png'
diagnosis(image)

1/1 [==============================] - 0s 142ms/step


'Covid Presence: positive.'

In [ ]:
# Define a function suitable for gradio
def gradio_fn(img_array):
    # Convert the numpy array to a PIL Image object
    img = Image.fromarray(np.uint8(img_array))

    # Convert grayscale image to RGB
    if img.mode != 'RGB':
        img = img.convert('RGB')

    # Resize the image to match the model's expected input size
    img = img.resize((256, 256))

    # Normalize the image data
    img_array = np.array(img) / 255.0

    # Expand the dimensions to match the model's expected input shape
    img_array = np.expand_dims(img_array, axis=0)

    # Use the model to predict
    prediction = model.predict(img_array)

    # Check if prediction is positive or negative
    if prediction <= 0.5:
        return 'Tumor presence: positive.'
    else:
        return 'Tumor presence: negative.'

In [ ]:
# Define the input component for the Gradio interface
inputs = gr.inputs.Image(shape=(256, 256))

# Define the output component for the Gradio interface
outputs = gr.outputs.Label()

# Create the Gradio interface
gr.Interface(gradio_fn, inputs, outputs, capture_session=True,share=True).launch(debug='True')